In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import math
import seaborn as sns

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
sklearn.__version__

'0.21.2'

## LendingClub Data Acquisition

In [3]:
# raw_q12019 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2019Q1.csv',header=1)
# raw_q42018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q4.csv',header=1)
# raw_q32018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q3.csv',header=1)
# raw_q22018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q2.csv',header=1)
# raw_q12018 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2018Q1.csv',header=1)
raw_q42017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q4.csv',header=1)
raw_q32017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q3.csv',header=1)
raw_q22017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q2.csv',header=1)
raw_q12017 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2017Q1.csv',header=1)
raw_q42016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q4.csv',header=1)
raw_q32016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q3.csv',header=1)
raw_q22016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q2.csv',header=1)
raw_q12016 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2016Q1.csv',header=1)
raw_2015 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2015.csv',header=1)
raw_2014 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2014.csv',header=1)
raw_2012to2013 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2012to2013.csv',header=1)
raw_2007to2011 = pd.read_csv('~/peervest/lc_data/LoanStats_securev1_2007to2011.csv',header=1)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,118) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,19,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/

In [4]:
# print(raw_q12019.shape)
# print(raw_q42018.shape)
# print(raw_q32018.shape)
# print(raw_q22018.shape)
# print(raw_q12018.shape)
print(raw_q42017.shape)
print(raw_q32017.shape)
print(raw_q22017.shape)
print(raw_q12017.shape)
print(raw_q42016.shape)
print(raw_q32016.shape)
print(raw_q22016.shape)
print(raw_q12016.shape)
print(raw_2015.shape)
print(raw_2014.shape)
print(raw_2012to2013.shape)
print(raw_2007to2011.shape)
#raw_q12019,raw_q42018,raw_q32018,raw_q22018,raw_q12018,
dfs = [raw_q42017,raw_q32017,raw_q22017,raw_q12017,
       raw_q42016,raw_q32016,raw_q22016,raw_q12016,raw_2015,
       raw_2014,raw_2012to2013, raw_2007to2011]

(118650, 150)
(122703, 150)
(105453, 150)
(96781, 150)
(103548, 150)
(99122, 150)
(97856, 150)
(133889, 150)
(421097, 150)
(235631, 150)
(188183, 150)
(42538, 150)


In [5]:
raw_lc_df = pd.concat(dfs, ignore_index=True)

In [6]:
raw_lc_df['loan_status'].value_counts()

Fully Paid                                             1076208
Current                                                 399465
Charged Off                                             268335
Late (31-120 days)                                       12138
In Grace Period                                           3934
Late (16-30 days)                                         2586
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     11
Name: loan_status, dtype: int64

In [7]:
raw_lc_df.shape

(1765451, 150)

## Data Cleaning

In [8]:
#FILTER OUT WHOSE STATUS IS NOT YET FINALIZED
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'Current']
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'Late (31-120 days)']
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'Late (16-30 days)']
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'In Grace Period']
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'Does not meet the credit policy. Status:Fully Paid']
raw_lc_df = raw_lc_df[raw_lc_df['loan_status'] != 'Does not meet the credit policy. Status:Charged Off']

#BINARIZE CLASSES
raw_lc_df['loan_status'] = raw_lc_df['loan_status'].map({'Charged Off': 0, 'Default': 0, 'Fully Paid': 1})

In [9]:
raw_lc_df['earliest_cr_line'] = pd.to_timedelta(pd.to_datetime(raw_lc_df['earliest_cr_line'])).dt.days

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  """Entry point for launching an IPython kernel.


In [10]:
def parse_percentage(percentage):
    '''Change percentage features into floats'''
    if str(percentage) == 'nan': return math.nan
    new = percentage.replace('%', '')
    return float(new) / 100.0

raw_lc_df['revol_util'] = raw_lc_df['revol_util'].apply(parse_percentage)
raw_lc_df['int_rate'] = raw_lc_df['int_rate'].apply(parse_percentage)

In [11]:
#FILTER OUT COLUMNS THAT ARE OBVIOUSLY REDUNDANT OR UNHELPFUL
lc_df = raw_lc_df[['loan_amnt', 'funded_amnt',
                'term', 'int_rate', 'installment',
                'emp_length', 'home_ownership', 'annual_inc',
                'verification_status', 'loan_status',
                'purpose', 'zip_code', 'addr_state', 'dti',
                'delinq_2yrs', 'earliest_cr_line', 'fico_range_low',
                'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq',
                'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
                'revol_util', 'total_acc', 'out_prncp',
                'out_prncp_inv', 'last_fico_range_high', 'last_fico_range_low',
                'collections_12_mths_ex_med', 'mths_since_last_major_derog',
                'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
                'acc_now_delinq', 'tot_coll_amt',
                'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
                'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
                'open_rv_12m', 'open_rv_24m', 'all_util',
                'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
                'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
                'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
                'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
                'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
                'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
                'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
                'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
                'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
                'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m',
                'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75',
                'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim',
                'total_bal_ex_mort', 'total_bc_limit',
                'total_il_high_credit_limit','grade', 'collection_recovery_fee', 
                'total_rec_prncp', 'title', 'total_rec_int', 'total_rec_late_fee', 
                'sub_grade', 'debt_settlement_flag', 'emp_title','issue_d','last_pymnt_d']]

In [12]:
lc_df.shape

(1344579, 101)

In [13]:
lc_df = lc_df.dropna(axis=0, subset=['loan_amnt','inq_last_6mths']).reset_index(drop=True)

In [14]:
#CAST DATATYPES
dtype = {
      'loan_amnt': 'int64',
      'term': 'object',
      'int_rate': 'float64',
      'emp_length': 'object',
      'home_ownership': 'object',
      'annual_inc': 'float64',
      'verification_status': 'object',
      'loan_status': 'object',
      'purpose': 'object',
      'zip_code': 'object',
      'addr_state': 'object',
      'dti': 'float64',
      'delinq_2yrs': 'int64',
      'fico_range_low': 'int64',
      'fico_range_high': 'int64',
      'inq_last_6mths': 'int64',
      'open_acc': 'int64',
      'pub_rec': 'int64',
      'revol_bal': 'int64',
      'total_acc': 'int64',
      'application_type': 'object'
    }
lc_df = lc_df.astype(dtype=dtype)

In [15]:
lc_df.head().T

,0,1,2,3,4
loan_amnt,30000,13000,15000,15750,19800
funded_amnt,30000,13000,15000,15750,19800
term,60 months,60 months,36 months,36 months,60 months
int_rate,0.3079,0.1505,0.1042,0.0735,0.1505
installment,985.21,309.62,486.98,488.84,471.57
emp_length,10+ years,10+ years,3 years,10+ years,3 years
home_ownership,RENT,RENT,RENT,RENT,RENT
annual_inc,40000,72000,55000,96500,108600
verification_status,Source Verified,Source Verified,Source Verified,Source Verified,Not Verified
loan_status,0,1,1,1,1


In [16]:
lc_df.columns[lc_df.isna().all()].tolist()

[]

In [17]:
lc_df.shape

(1344553, 101)

In [18]:
lc_df[['addr_state','annual_inc','collection_recovery_fee','emp_title',
           'fico_range_high','fico_range_low','funded_amnt','grade','home_ownership',
           'int_rate','loan_amnt','loan_status','purpose','sub_grade','title','total_rec_int',
           'total_rec_late_fee','total_rec_prncp','zip_code','debt_settlement_flag','out_prncp',
           'collection_recovery_fee']].head(20)
   

,addr_state,annual_inc,collection_recovery_fee,emp_title,fico_range_high,fico_range_low,funded_amnt,grade,home_ownership,int_rate,...,purpose,sub_grade,title,total_rec_int,total_rec_late_fee,total_rec_prncp,zip_code,debt_settlement_flag,out_prncp,collection_recovery_fee
0,NY,40000.0,0.0000,Supervsir,704,700,30000.0,G,RENT,0.3079,...,other,G1,Other,9463.05,98.52,2196.71,103xx,N,0.0,0.0000
1,FL,72000.0,0.0000,Sales manager,709,705,13000.0,C,RENT,0.1505,...,debt_consolidation,C4,Debt consolidation,598.13,0.00,13000.00,333xx,N,0.0,0.0000
2,WA,55000.0,0.0000,Finance,674,670,15000.0,B,RENT,0.1042,...,debt_consolidation,B3,Debt consolidation,1065.25,0.00,15000.00,981xx,N,0.0,0.0000
3,HI,96500.0,0.0000,Service Coordinator,709,705,15750.0,A,RENT,0.0735,...,debt_consolidation,A4,Debt consolidation,641.95,0.00,15750.00,968xx,N,0.0,0.0000
4,MA,108600.0,0.0000,engineer,729,725,19800.0,C,RENT,0.1505,...,credit_card,C4,Credit card refinancing,2471.93,0.00,19800.00,021xx,N,0.0,0.0000
5,NJ,100000.0,0.0000,Paralegal,779,775,10000.0,A,MORTGAGE,0.0532,...,debt_consolidation,A1,Debt consolidation,78.81,0.00,10000.00,087xx,N,0.0,0.0000
6,CA,55000.0,0.0000,Admissions Advisor,664,660,32600.0,B,RENT,0.1199,...,house,B5,Home buying,4036.10,0.00,32600.00,958xx,N,0.0,0.0000
7,NJ,54000.0,0.0000,Electro Mechanical Technician,669,665,10000.0,B,RENT,0.0944,...,debt_consolidation,B1,Debt consolidation,696.77,0.00,10000.00,070xx,N,0.0,0.0000
8,IN,85000.0,0.0000,Special Education Teacher,664,660,13000.0,B,MORTGAGE,0.0993,...,debt_consolidation,B2,Debt consolidation,569.54,0.00,13000.00,462xx,N,0.0,0.0000
9,FL,70000.0,200.5740,RN,694,690,10000.0,A,MORTGAGE,0.0672,...,other,A3,Other,371.08,46.14,1788.05,327xx,N,0.0,200.5740


In [19]:
#Change dtype
lc_df.loc[lc_df['emp_length'] == '< 1 year','emp_length'] = '0'
lc_df.loc[lc_df['emp_length'] == '10+ years', 'emp_length'] = '10'
lc_df['emp_length'] = lc_df['emp_length'].str[:1]
lc_df['emp_length'] = lc_df['emp_length'].astype(float)

In [20]:
#Drop nulls in zip & emp_title
lc_df = lc_df[lc_df['zip_code'].notnull()].reset_index(drop=True)
lc_df = lc_df[lc_df['emp_title'].notnull()].reset_index(drop=True)

In [21]:
counts = lc_df['emp_title'].value_counts()
idx = counts[counts.lt(1600)].index

lc_df.loc[lc_df['emp_title'].isin(idx) == False, 'emp_title_2'] = lc_df['emp_title']
lc_df.loc[lc_df['emp_title'].isin(idx), 'emp_title_2'] = 'Other'

In [22]:
clean_lc_df = lc_df.copy()

In [23]:
clean_lc_df.info(verbose=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260156 entries, 0 to 1260155
Data columns (total 102 columns):
loan_amnt                         int64
funded_amnt                       float64
term                              object
int_rate                          float64
installment                       float64
emp_length                        float64
home_ownership                    object
annual_inc                        float64
verification_status               object
loan_status                       object
purpose                           object
zip_code                          object
addr_state                        object
dti                               float64
delinq_2yrs                       int64
earliest_cr_line                  float64
fico_range_low                    int64
fico_range_high                   int64
inq_last_6mths                    int64
mths_since_last_delinq            float64
mths_since_last_record            float64
open_acc             

In [24]:
clean_lc_df = clean_lc_df.dropna(subset=
                                 ['collections_12_mths_ex_med','chargeoff_within_12_mths','last_pymnt_d'],axis=0)

### Feature Preprocessing

### TRAIN-TEST-SPLIT

In [25]:
X = clean_lc_df.drop(columns=['loan_status'])
y = clean_lc_df[['loan_status']]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,test_size=0.3)

#### Mean, Max, Zero Impute & Drops for Missing Values

In [27]:
#replace NaNs with column max, column means, or zeros where it makes sense to
nan_max_cols = [
    'mths_since_last_delinq',
    'mths_since_last_record',
    'mths_since_last_major_derog',
    'mths_since_rcnt_il',
    'mths_since_recent_bc',
    'mths_since_recent_bc_dlq',
    'mths_since_recent_inq',
    'mths_since_recent_revol_delinq',
    'mo_sin_old_il_acct',
    'mo_sin_old_rev_tl_op',
    'mo_sin_rcnt_rev_tl_op',
    'mo_sin_rcnt_tl',
]

nan_zero_cols = [
    'tot_coll_amt',
    'tot_cur_bal',
    'open_acc_6m',
    'open_act_il',
    'open_il_12m',
    'open_il_24m',
    'total_bal_il',
    'open_rv_12m',
    'open_rv_24m',
    'inq_fi',
    'inq_last_12m',
    'num_accts_ever_120_pd',
    'num_actv_bc_tl',
    'num_actv_rev_tl',
    'num_bc_sats',
    'num_bc_tl',
    'num_il_tl',
    'num_op_rev_tl',
    'num_rev_accts',
    'num_rev_tl_bal_gt_0',
    'num_sats',
    'num_tl_120dpd_2m',
    'num_tl_30dpd',
    'num_tl_90g_dpd_24m',
    'num_tl_op_past_12m',
    'mort_acc',
    'percent_bc_gt_75',
    'tot_hi_cred_lim',
    'total_bal_ex_mort',
    'total_bc_limit',
    'total_il_high_credit_limit',
    'bc_open_to_buy',
    'acc_open_past_24mths',
    'total_cu_tl',
    'total_rev_hi_lim',
    'emp_length',
    'pub_rec_bankruptcies',
    'tax_liens'
]

nan_mean_cols = [
    'dti',
    'annual_inc_joint',
    'dti_joint',
    'il_util',
    'all_util',
    'bc_util',
    'pct_tl_nvr_dlq',
    'avg_cur_bal',
    'revol_util',
]


for col in X_train:
    dt = X_train[col].dtype
    #FILL MISSING WITH MEAN from X_train, in both X_train & X_test
    if col in nan_mean_cols:
        X_train[col] = X_train[col].fillna(np.nanmean(X_train[col].values))
        X_test[col] = X_test[col].fillna(np.nanmean(X_train[col].values))
    #FILL MISSING WITH ZEROS from X_train, in both X_train & X_test
    if col in nan_zero_cols:
        X_train[col] = X_train[col].fillna(0.0)
        X_test[col] = X_test[col].fillna(0.0)
    #FILL MISSING WITH MAX from X_train, in both X_train & X_test
    if col in nan_max_cols:
        X_train[col] = X_train[col].fillna(np.nanmax(X_train[col].values) * 5)
        X_test[col] = X_test[col].fillna(np.nanmax(X_train[col].values) * 5)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [28]:
X_train.drop(columns=['title'],inplace=True)
X_test.drop(columns=['title'],inplace=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [29]:
null_columns=X_train.columns[X_train.isnull().any()]
X_train[null_columns].isnull().sum()

Series([], dtype: float64)

In [30]:
null_columns=X_test.columns[X_test.isnull().any()]
X_test[null_columns].isnull().sum()

Series([], dtype: float64)

### Categorical Variable One-Hot Encoding

In [31]:
print(clean_lc_df.shape)
print(X_train.shape)
print(X_test.shape)

(1258501, 102)
(880950, 100)
(377551, 100)


###### One-Hot Encode "term"

In [32]:
X_train['term'].value_counts()

 36 months    664098
 60 months    216852
Name: term, dtype: int64

In [33]:
#term
encoder = OneHotEncoder(categories='auto',handle_unknown='ignore').fit(X_train[['term']])
encoder.categories_

[array([' 36 months', ' 60 months'], dtype=object)]

In [34]:
ohe_term = pd.DataFrame(encoder.transform(X_train[['term']]).toarray(),
                        columns=encoder.get_feature_names(["term"]))
ohe_term_test = pd.DataFrame(encoder.transform(X_test[['term']]).toarray(),
                             columns=encoder.get_feature_names(["term"]))

ohe_term.shape

(880950, 2)

In [35]:
null_columns=ohe_term.columns[ohe_term.isnull().any()]
ohe_term[null_columns].isnull().sum()

Series([], dtype: float64)

###### One-Hot Encode "home_ownership"

In [36]:
X_train['home_ownership'].value_counts()

MORTGAGE    437095
RENT        354276
OWN          89260
ANY            176
OTHER          111
NONE            32
Name: home_ownership, dtype: int64

In [37]:
encoder = OneHotEncoder(categories='auto',handle_unknown='ignore').fit(X_train[['home_ownership']])
ohe_home_ownership = pd.DataFrame(encoder.transform(X_train[['home_ownership']]).toarray(),
                                  columns=encoder.get_feature_names(["home_ownership"]))
ohe_home_ownership_test = pd.DataFrame(encoder.transform(X_test[['home_ownership']]).toarray(),
                                       columns=encoder.get_feature_names(["home_ownership"]))

ohe_home_ownership.shape

(880950, 6)

In [38]:
null_columns=ohe_home_ownership.columns[ohe_home_ownership.isnull().any()]
ohe_home_ownership[null_columns].isnull().sum()

Series([], dtype: float64)

##### One-Hot Encode "purpose"

In [39]:
X_train['purpose'].value_counts()

debt_consolidation    515636
credit_card           193927
home_improvement       56020
other                  49167
major_purchase         18846
small_business          9837
medical                 9739
car                     9350
moving                  6120
vacation                5753
house                   4205
wedding                 1530
renewable_energy         593
educational              227
Name: purpose, dtype: int64

In [40]:
encoder = OneHotEncoder(categories='auto',handle_unknown='ignore').fit(X_train[['purpose']])
ohe_purpose = pd.DataFrame(encoder.transform(X_train[['purpose']]).toarray(),
                           columns=encoder.get_feature_names(["purpose"]))
ohe_purpose_test = pd.DataFrame(encoder.transform(X_test[['purpose']]).toarray(),
                                columns=encoder.get_feature_names(["purpose"]))

ohe_purpose.shape

(880950, 14)

In [41]:
null_columns=ohe_purpose.columns[ohe_purpose.isnull().any()]
ohe_purpose[null_columns].isnull().sum()

Series([], dtype: float64)

##### One-Hot Encode "zip_code"

In [42]:
X_train['zip_code'].value_counts() #945 ZIP CODES

945xx    9905
750xx    9713
112xx    9160
606xx    8265
300xx    8008
331xx    7590
070xx    7257
100xx    7116
770xx    7059
900xx    7014
330xx    6664
891xx    6649
917xx    6362
104xx    6219
117xx    6144
852xx    5642
921xx    5509
926xx    5359
913xx    5230
925xx    5130
334xx    4886
113xx    4623
601xx    4618
481xx    4589
604xx    4502
021xx    4490
853xx    4442
760xx    4415
600xx    4380
080xx    4362
         ... 
516xx       1
568xx       1
009xx       1
202xx       1
513xx       1
862xx       1
889xx       1
522xx       1
093xx       1
506xx       1
892xx       1
511xx       1
555xx       1
817xx       1
507xx       1
348xx       1
509xx       1
643xx       1
504xx       1
888xx       1
938xx       1
901xx       1
896xx       1
698xx       1
521xx       1
694xx       1
964xx       1
205xx       1
340xx       1
269xx       1
Name: zip_code, Length: 930, dtype: int64

In [43]:
encoder = OneHotEncoder(categories='auto',handle_unknown='ignore').fit(X_train[['zip_code']])
ohe_zip_code = pd.DataFrame(encoder.transform(X_train[['zip_code']]).toarray(),
                            columns=encoder.get_feature_names(["zip_code"]))
ohe_zip_code_test = pd.DataFrame(encoder.transform(X_test[['zip_code']]).toarray(),
                                 columns=encoder.get_feature_names(["zip_code"]))

ohe_zip_code.shape

(880950, 930)

In [44]:
null_columns=ohe_zip_code.columns[ohe_zip_code.isnull().any()]
ohe_zip_code[null_columns].isnull().sum()

Series([], dtype: float64)

##### One-Hot Encode "addr_state"

In [45]:
X_train['addr_state'].value_counts()

CA    128889
TX     73175
NY     72355
FL     61290
IL     34480
NJ     32377
PA     29962
OH     28671
GA     28397
VA     25175
NC     24495
MI     22352
MA     20772
AZ     20728
MD     20628
CO     19477
WA     19016
MN     16005
IN     14359
MO     13931
TN     13257
CT     13159
NV     12646
WI     11745
AL     10716
OR     10555
SC     10246
LA     10210
KY      8306
OK      7984
KS      7384
UT      6527
AR      6339
NM      4746
HI      4512
NH      4283
MS      4277
RI      3883
WV      3045
MT      2471
DE      2398
NE      2337
DC      2316
AK      2097
WY      1964
SD      1817
VT      1761
ME      1299
ND      1124
ID      1006
IA         6
Name: addr_state, dtype: int64

In [46]:
encoder = OneHotEncoder(categories='auto',handle_unknown='ignore').fit(X_train[['addr_state']])
ohe_addr_state = pd.DataFrame(encoder.transform(X_train[['addr_state']]).toarray(),
                              columns=encoder.get_feature_names(["addr_state"]))
ohe_addr_state_test = pd.DataFrame(encoder.transform(X_test[['addr_state']]).toarray(),
                                   columns=encoder.get_feature_names(["addr_state"]))

ohe_addr_state.shape

(880950, 51)

In [47]:
null_columns=ohe_addr_state.columns[ohe_addr_state.isnull().any()]
ohe_addr_state[null_columns].isnull().sum()

Series([], dtype: float64)

##### One-Hot Encode "application_type"

In [48]:
X_train['application_type'].value_counts()

Individual    868313
Joint App      12637
Name: application_type, dtype: int64

In [49]:
encoder = OneHotEncoder(categories='auto',handle_unknown='ignore').fit(X_train[['application_type']])
ohe_application_type = pd.DataFrame(encoder.transform(X_train[['application_type']]).toarray(),
                                    columns=encoder.get_feature_names(["application_type"]))
ohe_application_type_test = pd.DataFrame(encoder.transform(X_test[['application_type']]).toarray(),
                                    columns=encoder.get_feature_names(["application_type"]))

ohe_application_type.shape

(880950, 2)

In [50]:
null_columns=ohe_application_type.columns[ohe_application_type.isnull().any()]
ohe_application_type[null_columns].isnull().sum()

Series([], dtype: float64)

##### One-Hot Encode "grade"

In [51]:
X_train['grade'].value_counts()

B    258330
C    251178
A    153700
D    129383
E     61236
F     21026
G      6097
Name: grade, dtype: int64

In [52]:
encoder = OneHotEncoder(categories='auto',handle_unknown='ignore').fit(X_train[['grade']])
ohe_grade = pd.DataFrame(encoder.transform(X_train[['grade']]).toarray(),
                         columns=encoder.get_feature_names(["grade"]))
ohe_grade_test = pd.DataFrame(encoder.transform(X_test[['grade']]).toarray(),
                              columns=encoder.get_feature_names(["grade"]))
ohe_grade.shape

(880950, 7)

In [53]:
null_columns=ohe_grade.columns[ohe_grade.isnull().any()]
ohe_grade[null_columns].isnull().sum()

Series([], dtype: float64)

##### One-Hot Encode "sub_grade"

In [54]:
X_train['sub_grade'].value_counts()

C1    56420
B4    54578
B3    54314
B5    54225
C2    52340
C3    49256
C4    48864
B2    48538
B1    46675
C5    44298
A5    41920
A4    33796
D1    33226
A1    29711
D2    28852
D3    25301
A3    24305
A2    23968
D4    22929
D5    19075
E1    15609
E2    14035
E3    12137
E4    10361
E5     9094
F1     6469
F2     4726
F3     4019
F4     3202
F5     2610
G1     1960
G2     1465
G3     1066
G4      862
G5      744
Name: sub_grade, dtype: int64

In [55]:
encoder = OneHotEncoder(categories='auto',handle_unknown='ignore').fit(X_train[['sub_grade']])
ohe_sub_grade = pd.DataFrame(encoder.transform(X_train[['sub_grade']]).toarray(),
                             columns=encoder.get_feature_names(["sub_grade"]))
ohe_sub_grade_test = pd.DataFrame(encoder.transform(X_test[['sub_grade']]).toarray(),
                                  columns=encoder.get_feature_names(["sub_grade"]))

ohe_sub_grade.shape

(880950, 35)

In [56]:
null_columns=ohe_sub_grade.columns[ohe_sub_grade.isnull().any()]
ohe_sub_grade[null_columns].isnull().sum()

Series([], dtype: float64)

##### One-Hot Encode "emp_title"

In [57]:
len(X_train['emp_title_2'].value_counts())

50

In [58]:
encoder = OneHotEncoder(categories='auto',handle_unknown='ignore').fit(X_train[['emp_title_2']])
ohe_emp_title_2 = pd.DataFrame(encoder.transform(X_train[['emp_title_2']]).toarray(),
                               columns=encoder.get_feature_names(["emp_title_2"]))
ohe_emp_title_2_test = pd.DataFrame(encoder.transform(X_test[['emp_title_2']]).toarray(),
                                    columns=encoder.get_feature_names(["emp_title_2"]))

ohe_emp_title_2.shape

(880950, 50)

In [59]:
null_columns=ohe_emp_title_2.columns[ohe_emp_title_2.isnull().any()]
ohe_emp_title_2[null_columns].isnull().sum()

Series([], dtype: float64)

In [60]:
X_train.shape

(880950, 100)

In [61]:
ohe_emp_title_2.shape

(880950, 50)

In [62]:
X_train.reset_index().head()

,index,loan_amnt,funded_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,...,collection_recovery_fee,total_rec_prncp,total_rec_int,total_rec_late_fee,sub_grade,debt_settlement_flag,emp_title,issue_d,last_pymnt_d,emp_title_2
0,798015,20000,20000.0,60 months,0.1239,448.85,1.0,MORTGAGE,62000.0,Source Verified,...,0.000,20000.0,4704.74,0.00,C1,N,driver,Feb-2015,May-2017,driver
1,962964,15000,15000.0,36 months,0.1299,505.34,1.0,OWN,55000.0,Verified,...,207.693,6863.3,2712.89,50.54,C1,N,Teacher,Jun-2014,May-2016,Teacher
2,534392,16000,16000.0,36 months,0.0818,502.72,8.0,MORTGAGE,72000.0,Not Verified,...,0.000,16000.0,1974.15,75.42,B1,N,minister,Nov-2015,Jan-2019,Other
3,668134,24000,24000.0,36 months,0.0818,754.07,1.0,MORTGAGE,70000.0,Source Verified,...,0.000,24000.0,3124.60,0.00,B1,N,Southeast Retail Manager,Jul-2015,Jul-2018,Other
4,1068586,24000,24000.0,36 months,0.0662,736.89,1.0,MORTGAGE,175000.0,Source Verified,...,0.000,24000.0,2527.95,0.00,A2,N,IT Manager,Nov-2013,Nov-2016,Other


In [63]:
ohe_sub_grade.head()

,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,...,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
ohe_emp_title_2.head()

,emp_title_2_Account Executive,emp_title_2_Account Manager,emp_title_2_Accountant,emp_title_2_Administrative Assistant,emp_title_2_Administrator,emp_title_2_Analyst,emp_title_2_Assistant Manager,emp_title_2_Attorney,emp_title_2_Branch Manager,emp_title_2_CEO,...,emp_title_2_Technician,emp_title_2_Truck Driver,emp_title_2_Vice President,emp_title_2_driver,emp_title_2_manager,emp_title_2_owner,emp_title_2_sales,emp_title_2_supervisor,emp_title_2_teacher,emp_title_2_truck driver
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### One Hot Encode All Categorical Features

In [65]:
# encoder = OneHotEncoder(categories=['home_ownership','purpose','zip_code','sub_grade','emp_title_2'],
#                         handle_unknown='ignore').fit(X_train)
# ohe_all = pd.DataFrame(encoder.transform(X_train).toarray(),
#                        columns=encoder.get_feature_names(
#                            ['home_ownership','purpose','zip_code','sub_grade','emp_title_2']))
# ohe_all_test = pd.DataFrame(encoder.transform(X_train).toarray(),
#                             columns=encoder.get_feature_names(
#                                 ['home_ownership','purpose','zip_code','sub_grade','emp_title_2']))

### Join X_train with new One-Hot-Encoded TRAIN DataFrames 
##### & Join X_test with new One-Hot Encoded TEST DataFrames
- X_train + ohe_home_ownership + ohe_purpose + ohe_zip_code + ohe_application_type + ohe_sub_grade(more granular than ohe_grade) + ohe_emp_title_2
- I'm opting for Zip Code (944 cols) instead of State (50 cols - D.C. counts) in the first model under the hypothesis that the more granular location-based variable (zip) will reveal a more meaningful relationship than the broader borders of State

In [66]:
#Individual DF shapes
print(X_train.shape)
print(ohe_home_ownership.shape)
print(ohe_purpose.shape)
print(ohe_zip_code.shape)
print(ohe_addr_state.shape)
print(ohe_sub_grade.shape)
print(ohe_emp_title_2.shape)

(880950, 100)
(880950, 6)
(880950, 14)
(880950, 930)
(880950, 51)
(880950, 35)
(880950, 50)


In [67]:
type(ohe_emp_title_2)

pandas.core.frame.DataFrame

In [69]:
#Future Shape
print(X_train.shape[1]+ohe_home_ownership.shape[1]+ohe_purpose.shape[1]+
      ohe_zip_code.shape[1]+ohe_sub_grade.shape[1]+
     ohe_emp_title_2.shape[1])

1135


In [70]:
#concat_test_dfs = [X_train, ohe_home_ownership, ohe_purpose, ohe_zip_code,
#                   ohe_application_type, ohe_sub_grade, ohe_emp_title_2]
X_train_comb = X_train.reset_index().copy()
X_train_comb = pd.concat([X_train_comb.drop("home_ownership", axis=1), ohe_home_ownership], axis=1)

In [71]:
X_train_comb = pd.concat([X_train_comb.drop("purpose", axis=1), ohe_purpose], axis=1)

In [72]:
X_train_comb = pd.concat([X_train_comb.drop("zip_code", axis=1), ohe_zip_code], axis=1)

In [73]:
X_train_comb = pd.concat([X_train_comb.drop("application_type", axis=1), ohe_application_type], axis=1)

In [74]:
X_train_comb = pd.concat([X_train_comb.drop("sub_grade", axis=1), ohe_sub_grade], axis=1)

In [75]:
X_train_comb = pd.concat([X_train_comb.drop("emp_title_2", axis=1), ohe_emp_title_2], axis=1)

In [80]:
null_columns=X_train_comb.columns[X_train_comb.isnull().any()]
X_train_comb[null_columns].isnull().sum()

Series([], dtype: float64)

In [76]:
X_train_comb.shape

(880950, 1132)

In [77]:
#concat_test_dfs = [X_test, ohe_home_ownership_test, ohe_purpose_test, ohe_zip_code_test,
#                   ohe_application_type_test, ohe_sub_grade_test, ohe_emp_title_2_test]
X_test_comb = X_test.reset_index().copy()
X_test_comb = pd.concat([X_test_comb.drop("home_ownership", axis=1), ohe_home_ownership_test], axis=1)

In [78]:
X_test_comb = pd.concat([X_test_comb.drop("purpose", axis=1), ohe_purpose_test], axis=1)

In [79]:
X_test_comb = pd.concat([X_test_comb.drop("zip_code", axis=1), ohe_zip_code_test], axis=1)

In [80]:
X_test_comb = pd.concat([X_test_comb.drop("application_type", axis=1), ohe_application_type_test], axis=1)

In [81]:
X_test_comb = pd.concat([X_test_comb.drop("sub_grade", axis=1), ohe_sub_grade_test], axis=1)

In [82]:
X_test_comb = pd.concat([X_test_comb.drop("emp_title_2", axis=1), ohe_emp_title_2_test], axis=1)

In [88]:
null_columns=X_test_comb.columns[X_test_comb.isnull().any()]
X_test_comb[null_columns].isnull().sum()

Series([], dtype: float64)

### Reinstate X_train & X_test

In [83]:
X_test = X_test_comb

In [84]:
X_train = X_train_comb

### Save Each Set to a .TSV File

In [ ]:
#X_train.to_csv('X_train.tsv', sep='\t',chunksize=1000, compression='zip')

In [ ]:
#X_test.to_csv('X_test.tsv', sep='\t',chunksize=1000, compression='zip')

In [ ]:
#y_train.to_csv('y_train.tsv', sep='\t',chunksize=1000, compression='zip')

In [ ]:
#y_test.to_csv('y_test.tsv', sep='\t',chunksize=1000, compression='zip')

### Save Each Dataframe to a .PKL file

In [93]:
X_train.to_pickle('X_train.pkl')

In [94]:
X_test.to_pickle('X_test.pkl')

In [95]:
y_train.to_pickle('y_train.pkl')

In [96]:
y_test.to_pickle('y_test.pkl')

In [89]:
X_train.shape

(880950, 1132)

In [90]:
X_test.shape

(377551, 1132)

In [91]:
y_train.shape

(880950, 1)

In [92]:
y_test.shape

(377551, 1)